In [1]:
import os
import io
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import re
import subprocess
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append('../src')

In [2]:
all_data_root = '../../data/'

In [3]:
corpus = 'CMUMOSEI'
root_path = os.path.join(all_data_root, corpus, 'original_labels')

df_list = []
for ds in ['Train', 'Val', 'Test']:
    z = pd.read_csv(os.path.join(root_path, 'Data_{}_modified.csv'.format(ds)))
    z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']] = z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']].apply(lambda x: x if (x == 0).all() else x/x.sum(), axis=1)
    z['neutral'] = z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']].apply(lambda x: 1 if (x == 0).all() else 0, axis=1)
    z['sentiment'] = z['sentiment'].apply(lambda x: -1 if x < 0 else 1 if x > 0 else 0)
    z['subset'] = ds.lower().replace('val', 'dev')
    new_order = [0, 13, 1, 2, 3, 12, 4, 5, 6, 7, 8, 9, 10, 11]
    z = z[z.columns[new_order]]
    z = z.rename(columns={'video': 'video_name'})
    df_list.append(z.copy(deep=True))
    z.to_csv(os.path.join(all_data_root, corpus, 'labels', 'cmumosei_{}_multicorpus_labels.csv'.format(ds.lower().replace('val', 'dev'))), index=False)
    
new_df = pd.concat(df_list)
new_df.to_csv(os.path.join(all_data_root, corpus, 'cmumosei_multicorpus_labels.csv'), index=False)


In [4]:
corpus = 'CMUMOSEI'
root_path = os.path.join(all_data_root, corpus, 'original_labels')

df_list = []
for ds in ['Train', 'Val', 'Test']:
    z = pd.read_csv(os.path.join(root_path, 'Data_{}_modified.csv'.format(ds)))
    z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']] = z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']].apply(lambda x: np.where(x > 0, 1, 0))
    z['neutral'] = z[['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']].apply(lambda x: 1 if (x == 0).all() else 0, axis=1)
    z['sentiment'] = z['sentiment'].apply(lambda x: -1 if x < 0 else 1 if x > 0 else 0)
    z['subset'] = ds.lower().replace('val', 'dev')
    new_order = [0, 13, 1, 2, 3, 12, 4, 5, 6, 7, 8, 9, 10, 11]
    z = z[z.columns[new_order]]
    z = z.rename(columns={'video': 'video_name'})
    df_list.append(z.copy(deep=True))
    z.to_csv(os.path.join(all_data_root, corpus, 'labels', 'cmumosei_{}_singlecorpus_labels.csv'.format(ds.lower().replace('val', 'dev'))), index=False)
    
new_df = pd.concat(df_list)
new_df.to_csv(os.path.join(all_data_root, corpus, 'cmumosei_singlecorpus_labels.csv'), index=False)


In [5]:
z

,video_name,subset,start_time,end_time,sentiment,neutral,happy,sad,anger,surprise,disgust,fear,text,ASR
0,-6rXp3zJ3kc,test,14.468000,22.882000,0,0,0,1,1,0,0,0,And it's a retirement future that can ultimate...,and it's a retirement future that ultimately ...
1,-6rXp3zJ3kc,test,45.428000,58.130000,-1,0,0,1,0,0,0,0,Many people are not until they get to be a old...,aware of many people are not until i get to b...
2,-6rXp3zJ3kc,test,58.130000,71.960000,1,0,0,1,0,0,0,0,"So the answer to the question, can I contribut...",so the answer to the question can i contribut...
3,-9y-fZ3swSY,test,0.000000,6.368000,1,0,1,0,0,0,0,0,"is, you can say, hey I really like baby skin, ...",especially if i'd say hey can what i really l...
4,-9y-fZ3swSY,test,6.368000,11.785500,1,0,1,0,0,0,0,0,"It’s so nice, by giving a good example and not...",it's so nice by giving a good example and not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4648,zhNksSReaQk,test,39.412925,41.798639,0,0,1,0,0,0,0,1,I've never done this before,blog or never done this before
4649,zhNksSReaQk,test,41.876190,43.054649,0,1,0,0,0,0,0,0,"Um, I'm in my kitchen",i am a kitchen
4650,zhNksSReaQk,test,43.054649,46.067800,1,0,1,0,0,0,0,0,You can see all my carbonated water over there,i can see all my carbonated water
4651,zvZd3V5D5Ik,test,6.147000,11.776000,1,0,1,0,0,0,0,0,But I can bridge the gap of language because I...,but i can bridge the gap of language because ...


In [188]:
from sklearn.preprocessing import OneHotEncoder

corpus = 'RAMAS'
root_path = os.path.join(all_data_root, corpus, 'original_labels')

z = pd.read_csv(os.path.join(root_path, 'conf_4_RAMAS_final.csv'))


emotion_column = z['Emotion'].values
enc = OneHotEncoder()
a = enc.fit_transform(emotion_column.reshape(-1, 1)).toarray()
z = pd.concat([z, pd.DataFrame(a, columns=enc.categories_[0])], axis=1)

z['Sentiment_neg'] = z['Sentiment_neg'].apply(lambda x: -x)
z['Sentiment_neu'] = z['Sentiment_neu'].apply(lambda x: 0)
z['sentiment'] = z.loc[:, ['Sentiment_neg', 'Sentiment_neu', 'Sentiment_pos']].sum(axis=1)
z['Subset'] = z['Subset'].apply(lambda x: x.lower())

z = z.drop(['Emotion', 'Emo_happy', 'Emo_sad', 'Emo_anger', 'Emo_surprise',
            'Emo_disgust', 'Emo_fear', 'Sentiment_neg', 'Sentiment_neu', 'Sentiment_pos'], axis=1)

new_order = [0, 4, 3, 1, 2, 12, 8, 7, 9, 5, 11, 6, 10]
z = z[z.columns[new_order]]

z = z.rename(columns={'File': 'video_id',
                      'Name_video_cut': 'video_name',
                      'Subset': 'subset',
                      'Start': 'start_time',
                      'End': 'end_time',
                      'Neutral': 'neutral', 
                      'Happy': 'happy', 
                      'Sad': 'sad', 
                      'Angry': 'anger',
                      'Surprised': 'surprise', 
                      'Disgusted': 'disgust',
                      'Scared': 'fear'})

z.to_csv(os.path.join(all_data_root, corpus, 'ramas_labels.csv'), index=False)

In [185]:
z

,video_id,video_name,subset,start_time,end_time,sentiment,neutral,happy,sad,anger,surprise,disgust,fear
0,10dec_D11_1,10dec_D11_1__0000.mov,train,125.701,137.617,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10dec_D11_1,10dec_D11_1__0001.mov,train,139.831,142.400,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10dec_D11_1,10dec_D11_1__0002.mov,train,215.625,241.800,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10dec_D11_1,10dec_D11_1__0003.mov,train,245.125,257.460,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10dec_D11_2,10dec_D11_2__0004.mov,train,126.120,130.880,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,22dec_N16_1,22dec_N16_1__1802.mov,train,113.060,114.360,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1788,22dec_N16_2,22dec_N16_2__1803.mov,train,11.220,21.235,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1789,22dec_N16_2,22dec_N16_2__1804.mov,train,24.730,41.287,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1790,22dec_N16_2,22dec_N16_2__1805.mov,train,43.421,53.286,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.preprocessing import OneHotEncoder

corpus = 'MELD'
root_path = os.path.join(all_data_root, corpus, 'original_labels')

df_list = []
for ds in ['train', 'dev', 'test']:
    z = pd.read_csv(os.path.join(root_path, '{}_sent_emo.csv'.format(ds)))
    z['video_name'] = z.apply(lambda x: 'dia{}_utt{}'.format(x['Dialogue_ID'], x['Utterance_ID']), axis=1)
    z['Sentiment'] = z['Sentiment'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1}[x])
    
    emotion_column = z['Emotion'].values
    enc = OneHotEncoder()
    a = enc.fit_transform(emotion_column.reshape(-1, 1)).toarray()
    z = pd.concat([z, pd.DataFrame(a, columns=enc.categories_[0])], axis=1)
    
    z = z.drop(['Sr No.', 'Speaker', 'Emotion', 'Dialogue_ID', 'Utterance_ID',
            'Season', 'Episode'], axis=1)
    
    z['subset'] = ds
    
    new_order = [4, 12, 2, 3, 1, 9, 8, 10, 5, 11, 6, 7, 0]
    z = z[z.columns[new_order]]

    z = z.rename(columns={'StartTime': 'start_time',
                      'EndTime': 'end_time',
                      'Sentiment': 'sentiment',
                      'sadness': 'sad',
                      'joy': 'happy',
                      'Utterance': 'text'})
    if 'train' in ds:
        z = z.drop(z[z.video_name == 'dia125_utt3'].index)

    if 'dev' in ds:
        z = z.drop(z[z.video_name == 'dia110_utt7'].index)

    df_list.append(z.copy(deep=True))
    
    z.to_csv(os.path.join(all_data_root, corpus, 'labels', 'meld_{}_labels.csv'.format(ds.lower())), index=False)   
    
    
new_df = pd.concat(df_list)
new_df.to_csv(os.path.join(all_data_root, corpus, 'meld_labels.csv'), index=False)

In [7]:
z

,video_name,subset,start_time,end_time,sentiment,neutral,happy,sad,anger,surprise,disgust,fear,text
0,dia0_utt0,test,"00:14:38,127","00:14:40,378",1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Why do all youre coffee mugs have numbers on ...
1,dia0_utt1,test,"00:14:40,629","00:14:47,385",-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Oh. Thats so Monica can keep track. That way ...
2,dia0_utt2,test,"00:14:56,353","00:14:57,520",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Y'know what?
3,dia1_utt0,test,"0:10:44,769","0:10:46,146",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"Come on, Lydia, you can do it."
4,dia1_utt1,test,"0:10:46,146","0:10:46,833",1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Push!
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,dia279_utt11,test,"00:14:35,457","00:14:40,211",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"Yeah, I mean, come on Ross, no one will even n..."
2606,dia279_utt12,test,"00:14:42,256","00:14:43,840",-1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,They’re not listening too me?
2607,dia279_utt13,test,"00:14:44,008","00:14:48,511",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Of course they’re listening to you! Everybody ...
2608,dia279_utt14,test,"00:14:48,138","00:14:52,390",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Monica you really think I should try this phas...
